# Toolkits
工具箱是工具的集合，这些工具被设计成一起用于特定的任务。他们有方便的加载方法。有关可用的现成工具包的完整列表，请访问[Integrations](https://python.langchain.com/docs/integrations/toolkits/)

所有工具包都公开了一个`get_tools`法，该方法返回一个工具列表。因此你可以

```python
# Initialize a toolkit
toolkit = ExampleTookit(...)

# Get list of tools
tools = toolkit.get_tools()

# Create agent
agent = create_agent_method(llm, tools, prompt)
```